In [1]:
import random
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d

!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import colors as cc

from ipywidgets import widgets
from ipywidgets import interact, interact_manual, interactive

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Building jupyterlab assets (production, minimized)


In [73]:
x_bounds = np.array([0,13])
x_buffer = np.array([2,-2])

y_bounds = np.array([0,16])
y_buffer = np.array([2,-2])

x_plot = x_bounds + x_buffer
y_plot = y_bounds + y_buffer

In [87]:
def plotter_poly_art(num_points=1000, perc_fill=1, n_fill_lines=50, min_scale=0.1, guides=False):
    
    x = np.random.uniform(low=0, high=1, size=num_points).reshape(num_points, 1)*x_bounds[1]
    y = np.random.uniform(low=0, high=1, size=num_points).reshape(num_points, 1)*y_bounds[1]
    pts = np.hstack([x, y])
    
    vor = Voronoi(pts)
    verts = vor.vertices
    shapes_ind = vor.regions

    shapes_ind = [x + x[0:1] for i,x in enumerate(shapes_ind) if len(x) != 0 and -1 not in x]
    shapes = [verts[x] for i,x in enumerate(shapes_ind)]
    
    color_index = np.linspace(0,1,len(shapes))
    
    total_polys = len(shapes)
    filled_polys = int(perc_fill*total_polys)
    polygon_ind = random.sample(range(total_polys), filled_polys)

    for i in range(filled_polys):
        polygon = shapes[polygon_ind[i]]
    
        center = np.mean(polygon, axis=0)
        poly_center = polygon - center

        for j in np.linspace(min_scale, 1, num=n_fill_lines):
            scaled_poly = j*(poly_center)+center
            shapes.append(scaled_poly)
            color_index = np.append(color_index, color_index[i])
    
    color_index = np.array(color_index)
    
    fig, ax = plt.subplots(figsize=(20,20), facecolor='white')

    if not guides:
        plt.grid(False)
        plt.axis('off')
   
    ax.set_xlim(x_plot)
    ax.set_ylim(y_plot)
    # change colors by modifying the cmap reference below
    lc = LineCollection(shapes, array=color_index, cmap=plt.cm.spring)
    ax.add_collection(lc)
    
    return fig, ax

slider = interactive(plotter_poly_art,
                     num_points=(10,2000,10),
                     perc_fill=(0,1,0.05),
                     n_fill_lines=(1,50,1),
                     min_scale=(0,1,0.01))
display(slider)

interactive(children=(IntSlider(value=1000, description='num_points', max=2000, min=10, step=10), FloatSlider(…

In [89]:
fig, ax = slider.result
fig.savefig('voronoi_art_10.jpeg', bbox_inches = 'tight', pad_inches = 0.5)